In [1]:
import os
cwd = os.getcwd()

In [2]:
import sys
sys.path.insert(0, cwd + "/src")

In [3]:
from dendrotweaks.utils import *
from dendrotweaks.membrane.io.reader import Reader
from dendrotweaks.membrane.io.parser import Parser
from dendrotweaks.membrane.io.code_generators import PythonCodeGenerator

/home/roman/Documents/code/DendroTweaks/app/src


In [4]:
mechanism_name = "Kv"
archive_name = "Park_test"
path_to_mod_file = cwd + f"/data/mod/{archive_name}/{mechanism_name}.mod"
path_to_python_file = cwd + f"/data/mechanisms/{archive_name}/{mechanism_name}.py"
path_to_json_file = cwd + f"/data/json/{archive_name}/{mechanism_name}.json"
path_to_template = cwd + "/data/templates/template_dd_new.py"

# Read and preprocess

In [5]:
reader = Reader()

In [6]:
reader.read_file(path_to_mod_file)

In [7]:
reader.preprocess()

In [8]:
reader.split_content_in_blocks()

Split content into blocks:
    1 - TITLE
    1 - COMMENT
    1 - NEURON
    1 - UNITS
    1 - PARAMETER
    1 - ASSIGNED
    1 - STATE
    1 - BREAKPOINT
    1 - DERIVATIVE
    1 - INITIAL
    1 - FUNCTION
    1 - PROCEDURE
    0 - KINETIC
No unmatched content.


In [9]:
reader.blocks

{'TITLE': ['TITLE Kv_Park_ref\n'],
 'COMMENT': ['COMMENT\n26 Ago 2002 Modification of original channel to allow variable time step and to correct an initialization error.\n    Done by Michael Hines(michael.hines@yale.e) and Ruggero Scorcioni(rscorcio@gmu.edu) at EU Advance Course in Computational Neuroscience. Obidos, Portugal\n\nkv.mod\n\nPotassium channel, Hodgkin-Huxley style kinetics\nKinetic rates based roughly on Sah et al. and Hamill et al. (1991)\n\nAuthor\nENDCOMMENT'],
 'NEURON': ['NEURON {\n\tSUFFIX Kv\n\tUSEION k READ ek WRITE ik\n\tRANGE gbar, i, v12, q\n}'],
 'UNITS': ['UNITS {\n\t(mA) = (milliamp)\n\t(mV) = (millivolt)\n\t(S)  = (siemens)\n\t(um) = (micron)\n}'],
 'PARAMETER': ['PARAMETER {\n\tgbar = 0.0   (S/cm2)                                                                                      \n\tRa   = 0.02  (/mV/ms) \n\tRb   = 0.006 (/mV/ms) \n\tv12  = 25    (mV)\t  \n\tq    = 9     (mV)     \n\ttemp = 23    (degC)   \n\tq10  = 2.3   (1)\t  \n}'],
 'ASSIGNED': ['A

# Parse

In [10]:
parser = Parser()

In [11]:
parser.parse(reader.blocks)

Parsed TITLE block.
Parsed COMMENT block.
Parsed NEURON block.
Parsed UNITS block.
Parsed PARAMETER block.
Parsed ASSIGNED block.
Parsed STATE block.
Parsed BREAKPOINT block.
Parsed DERIVATIVE block.
Parsed INITIAL block.
Parsed FUNCTION block.
Parsed PROCEDURE block.


In [12]:
parser.postprocess()

Restoring expressions in BREAKPOINT
Restoring expressions in DERIVATIVE
Restoring expressions in FUNCTION
Restoring expressions in PROCEDURE


In [13]:
parser.ast

{'TITLE': ' Kv_Park_ref',
 'COMMENT': ['26 Ago 2002 Modification of original channel to allow variable '
             'time step and to correct an initialization error.',
             '    Done by Michael Hines(michael.hines@yale.e) and Ruggero '
             'Scorcioni(rscorcio@gmu.edu) at EU Advance Course in '
             'Computational Neuroscience. Obidos, Portugal',
             'kv.mod',
             'Potassium channel, Hodgkin-Huxley style kinetics',
             'Kinetic rates based roughly on Sah et al. and Hamill et al. '
             '(1991)',
             'Author'],
 'NEURON': {'suffix': 'Kv',
            'useion': [{'ion': 'k', 'read': ['ek'], 'write': ['ik']}],
            'range': ['gbar', 'i', 'v12', 'q']},
 'UNITS': {'mA': 'milliamp', 'mV': 'millivolt', 'S': 'siemens', 'um': 'micron'},
 'PARAMETER': [{'name': 'gbar', 'value': 0.0, 'unit': 'S/cm2'},
               {'name': 'Ra', 'value': 0.02, 'unit': '/mV/ms'},
               {'name': 'Rb', 'value': 0.006, 'unit': '/

In [14]:
import json

with open(path_to_json_file, 'w') as f:
    json.dump(parser._ast, f, indent=4)

# Generate python code

In [15]:
generator = PythonCodeGenerator()

In [16]:
generator.generate(parser.ast, path_to_template)

In [17]:
generator.write_file(path_to_python_file)

Saved content to /home/roman/Documents/code/DendroTweaks/app/src/data/mechanisms/Park_test/Kv.py
